## Lab journal for mini-project №2

### Downloading data

In [55]:
import sys
import numpy as np

We have downloaded data for our roommate using sratools-kit from here https://www.ncbi.nlm.nih.gov/sra/?term=SRR1705851
Reference genome have been taken from here http://public.dobzhanskycenter.ru/mrayko/Week2/KF848938.1.fasta via wget 

In [4]:
%ls

Lab_journal.ipynb  reference.fasta  roommate.fastq


### Aligning reads using bwa and samtools

In [16]:
!bwa index reference.fasta

[bwa_index] Pack FASTA... 0.00 sec
[bwa_index] Construct BWT for the packed sequence...
[bwa_index] 0.00 seconds elapse.
[bwa_index] Update BWT... 0.00 sec
[bwa_index] Pack forward-only FASTA... 0.00 sec
[bwa_index] Construct SA from BWT and Occ... 0.00 sec
[main] Version: 0.7.17-r1188
[main] CMD: bwa index reference.fasta
[main] Real time: 0.011 sec; CPU: 0.002 sec


In [23]:
!bwa mem reference.fasta roommate.fastq > alignment.sam

[M::bwa_idx_load_from_disk] read 0 ALT contigs
[M::process] read 68388 sequences (10000129 bp)...
[M::process] read 67628 sequences (10000233 bp)...
[M::mem_process_seqs] Processed 68388 reads in 1.686 CPU sec, 1.652 real sec
[M::process] read 67698 sequences (10000046 bp)...
[M::mem_process_seqs] Processed 67628 reads in 1.250 CPU sec, 1.194 real sec
[M::process] read 67652 sequences (10000169 bp)...
[M::mem_process_seqs] Processed 67698 reads in 1.697 CPU sec, 1.615 real sec
[M::process] read 68072 sequences (10000295 bp)...
[M::mem_process_seqs] Processed 67652 reads in 1.656 CPU sec, 1.594 real sec
[M::process] read 18827 sequences (2716992 bp)...
[M::mem_process_seqs] Processed 68072 reads in 1.436 CPU sec, 1.387 real sec
[M::mem_process_seqs] Processed 18827 reads in 0.656 CPU sec, 0.617 real sec
[main] Version: 0.7.17-r1188
[main] CMD: bwa mem reference.fasta roommate.fastq
[main] Real time: 9.639 sec; CPU: 8.494 sec


In [21]:
%ls

aligment.sam   Lab_journal.ipynb    reference.fasta.ann  reference.fasta.sa
alignment.bam  reference.fasta      reference.fasta.bwt  roommate.fastq
alligment.sam  reference.fasta.amb  reference.fasta.pac


In [24]:
!samtools view -S -b alignment.sam > alignment.bam

In [25]:
!samtools flagstat alignment.bam

361349 + 0 in total (QC-passed reads + QC-failed reads)
0 + 0 secondary
3084 + 0 supplementary
0 + 0 duplicates
361116 + 0 mapped (99.94% : N/A)
0 + 0 paired in sequencing
0 + 0 read1
0 + 0 read2
0 + 0 properly paired (N/A : N/A)
0 + 0 with itself and mate mapped
0 + 0 singletons (N/A : N/A)
0 + 0 with mate mapped to a different chr
0 + 0 with mate mapped to a different chr (mapQ>=5)


In [28]:
!samtools sort alignment.bam -o alignment_sorted.bam

In [29]:
!samtools index alignment_sorted.bam

### Making pileup and trying to find mutations

In [30]:
%ls

alignment.bam             my.mpileup           reference.fasta.fai
alignment.sam             reference.fasta      reference.fasta.pac
alignment_sorted.bam      reference.fasta.amb  reference.fasta.sa
alignment_sorted.bam.bai  reference.fasta.ann  roommate.fastq
Lab_journal.ipynb         reference.fasta.bwt


In [71]:
!samtools mpileup -f reference.fasta alignment_sorted.bam > roommate_genome.mpileup

[mpileup] 1 samples in 1 input files
<mpileup> Set max per-file depth to 8000


In [73]:
# Finding mutations with freq >= 0.95

!java -jar /home/belsawan/Downloads/VarScan.v2.4.0.jar mpileup2snp roommate_genome.mpileup \
--min-var-freq 0.95 --variants --output-vcf 1 > varscan_roommate_95_results.vcf

Only SNPs will be reported
Min coverage:	8
Min reads2:	2
Min var freq:	0.95
Min avg qual:	15
P-value thresh:	0.01
Reading input from roommate_genome.mpileup
1665 bases in pileup file
5 variant positions (5 SNP, 0 indel)
1 were failed by the strand-filter
4 variant positions reported (4 SNP, 0 indel)


In [110]:
!cat varscan_roommate_95_results.vcf | gawk 'NR>24 {print "Mutations:",  $1, $2, $4, $5, "| Freq in percent:", substr($10, 26, 5)}'

Mutations: KF848938.1 117 C T | Freq in percent: :99.2
Mutations: KF848938.1 774 T C | Freq in percent: 99.96
Mutations: KF848938.1 999 C T | Freq in percent: :99.8
Mutations: KF848938.1 1260 A C | Freq in percent: 99.96


* These are out mutations which needed to be analyzed (min-var-freq = 0.95)

In [77]:
# Finding mutations with freq >= 0.001

!java -jar /home/belsawan/Downloads/VarScan.v2.4.0.jar mpileup2snp roommate_genome.mpileup \
--min-var-freq 0.001 --variants --output-vcf 1 > varscan_roommate_01_results.vcf

Only SNPs will be reported
Min coverage:	8
Min reads2:	2
Min var freq:	0.001
Min avg qual:	15
P-value thresh:	0.01
Reading input from roommate_genome.mpileup
1665 bases in pileup file
12 variant positions (10 SNP, 2 indel)
1 were failed by the strand-filter
9 variant positions reported (9 SNP, 0 indel)


In [109]:
!cat varscan_roommate_01_results.vcf | gawk 'NR>24 {print "Mutations:",  $1, $2, $4, $5, "| Freq in percent:", substr($10, 26, 5)}'

Mutations: KF848938.1 117 C T | Freq in percent: :99.2
Mutations: KF848938.1 276 A G | Freq in percent: 0.27%
Mutations: KF848938.1 307 C T | Freq in percent: :0.76
Mutations: KF848938.1 774 T C | Freq in percent: 99.96
Mutations: KF848938.1 915 T C | Freq in percent: 0.3%:
Mutations: KF848938.1 999 C T | Freq in percent: :99.8
Mutations: KF848938.1 1260 A C | Freq in percent: 99.96
Mutations: KF848938.1 1458 T C | Freq in percent: :1.1%
Mutations: KF848938.1 1460 A G | Freq in percent: 0.28%


* These are out mutations which needed to be analyzed (min-var-freq = 0.001)

### Downloading fastq reads for another strains

In [46]:
!fastq-dump SRR1705859

Read 233327 spots for SRR1705859
Written 233327 spots for SRR1705859


In [47]:
!fastq-dump SRR1705860

Read 249964 spots for SRR1705860
Written 249964 spots for SRR1705860


In [48]:
!fastq-dump SRR1705858

Read 256586 spots for SRR1705858
Written 256586 spots for SRR1705858


### Making control analyzes

In [49]:
%ls

alignment.bam             reference.fasta.amb  SRR1705858.fastq
alignment.sam             reference.fasta.ann  SRR1705859.fastq
alignment_sorted.bam      reference.fasta.bwt  SRR1705860.fastq
alignment_sorted.bam.bai  reference.fasta.fai  varscan_01_results.vcf
Lab_journal.ipynb         reference.fasta.pac  varscan_95_results.vcf
my.mpileup                reference.fasta.sa
reference.fasta           roommate.fastq


In [50]:
# Number of lines

!wc -l SRR1705858.fastq

1026344 SRR1705858.fastq


In [51]:
# Number of lines

!wc -l SRR1705859.fastq

933308 SRR1705859.fastq


In [52]:
# Number of lines

!wc -l SRR1705860.fastq

999856 SRR1705860.fastq


In [87]:
# Counting

a = np.array([1026344./4, 933308./4, 999856./4])
print('Average number of reads:', round(a.mean(), 2))

ref = 'CAAAAACTTCCTGGAAATGACAACAGCACGGCAACGCTGTGCCTTGGGCACCATGCAGTGCCAAACGGAACAATAGTGAAAACAATCACGAATGACCAAATTGAAGTTACTAATGCCACTGAGCTGGTTCAGAGTTCCTCAACAGGTGAAATATGCAACAGTCCTCATCAGATCCTTGATGGAGAAAACTGCACACTAATAGATGCTCTATTGGGAGACCCTCAGTGTGATGGCTTCCAAAACAAGAAATGGGACCTTTTTGTTGAACGAAGCAAAGCCCACAGCAACTGTTACCCTTATGATGTGCCGGATTATGCCTCCCTTAGGTCACTAGTTGCCTCATCCGGCACACTGGAGTTTAACAATGAAAGCTTCAATTGGACTGGAGTCACTCAAAACGGAACAAGCTCTGCTTGCATAAGGAGATCTAATAATAGTTTCTTTAGTAGATTGAATTGGTTGACCCACTTAAACTTCAAATACCCAGCATTGAACGTGACTATGCCAAACAATGAACAATTTGACAAATTGTACATTTGGGGGGTTCACCACCCGGGTACGGACAAGGACCAAATCTTCCTGTATGCTCAAGCAGCAGGAAGAATCACAGTATCTACCAAAAGAAGCCAACAAGCTGTAATTCCGAATATCGGATCTAGACCCAGAGTAAGGAATATCCCTAGCAGAGTAAGCATCTATTGGACAATAGTAAAACCGGGAGACATACTTTTGATTAACAGCACAGGGAATCTAATTGCTCCTAGGGGTTACTTTAAAATACGAAGTGGGAAAAGCTCAATAATGAGATCAGATGCACCCATTGGCAAATGCAATTCTGCATGCATCACTCCAAATGGAAGCATTCCCAATGACAAACCATTCCAAAATGTAAACAGGATCACATACGGGGCCTGTCCCAGATATGTTAAGCAAAACACTCTGAAATTGGCAACAGGAATGAGAAATGTACCAGAGAAACAAACTAGAGGCATATTTGGCGCAATAGCTGGTTTCATAGAAAATGGTTGGGAGGGAATGGTGGATGGTTGGTACGGTTTCAGGCATCAAAATTCTGAGGGAAGGGGACAAGCAGCAGATCTCAAAAGCACTCAAGCAGCAATCGATCAAATCAATGGGAAGCTGAATAGATTGATCGGGAAAACCAACGAGAAATTCCATCAGATTGAAAAAGAATTCTCAGAAGTCGAAGGGAGAATTCAGGACCTTGAGAAATATGTTGAGGACACTAAAATAGATCTATGGTCATACAACGCGGAGCTTCTTGTTGCCCTGGAGAACCAACACACAATTGATCTAACTGACTCAGAAATGAACAAATTGTTTGAAAAAACAAAGAAGCAACTGAGGGAAAATGCTGAGGATATGGGCAATGGTTGTTTCAAAATATACCACAAATGTGACAATGCCTGCATAGGATCAATCAGAAATGGAACTTATGACCACGATGTGTACAGAGATGAAGCATTAAACAACCGATTCCAGATCAAGGGAGTTGAGCTGAAGTCAGGGTACAAAGATTGGATCCTATGGATTTCCTTTGCCATATCATGTTTTTTGCTTTGTGTTGCTTTGTTGGGGTTCATCATGTGGGCCTGCCAAAAAGGCAACATTAGGTGCAACATTTGCATTTGAGTGCATTAATTA'
print('Number of pairs in reference genome:', len(ref))

print('Assume avg len of read is 100, then there are', 246625/100, 'approximately reads for letter')

Average number of reads: 246625.67
Number of pairs in reference genome: 1665
Assume avg len of read is 100, then there are 2466.25 approximately reads for letter


#### We will use next abbs:
* strain1 = SRR1705858
* strain2 = SRR1705859
* strain3 = SRR1705860

In [69]:
!mv SRR1705858.fastq strain1.fastq
!mv SRR1705859.fastq strain2.fastq
!mv SRR1705860.fastq strain3.fastq

In [70]:
%ls

alignment.bam             reference.fasta.amb  strain1.fastq
alignment.sam             reference.fasta.ann  strain2.fastq
alignment_sorted.bam      reference.fasta.bwt  strain3.fastq
alignment_sorted.bam.bai  reference.fasta.fai  varscan_01_results.vcf
Lab_journal.ipynb         reference.fasta.pac  varscan_95_results.vcf
my.mpileup                reference.fasta.sa
reference.fasta           roommate.fastq


#### Analyze for strains

In [88]:
!samtools index alignment_sorted_strain1.bam
!samtools index alignment_sorted_strain2.bam
!samtools index alignment_sorted_strain3.bam

In [89]:
%ls

alignment.bam                     reference.fasta.pac
alignment.sam                     reference.fasta.sa
alignment_sorted.bam              roommate.fastq
alignment_sorted.bam.bai          roommate_genome.mpileup
alignment_sorted_strain1.bam      strain1.fastq
alignment_sorted_strain1.bam.bai  strain1_genome.mpileup
alignment_sorted_strain2.bam      strain2.fastq
alignment_sorted_strain2.bam.bai  strain2_genome.mpileup
alignment_sorted_strain3.bam      strain3.fastq
alignment_sorted_strain3.bam.bai  strain3_genome.mpileup
Lab_journal.ipynb                 varscan_roommate_01_results.vcf
reference.fasta                   varscan_roommate_95_results.vcf
reference.fasta.amb               varscan_strain_01_results.vcf
reference.fasta.ann               varscan_strain1_01_results.vcf
reference.fasta.bwt               varscan_strain2_01_results.vcf
reference.fasta.fai               varscan_strain3_01_results.vcf


In [111]:
# For strain 1

!samtools mpileup -f reference.fasta alignment_sorted_strain1.bam > strain1_genome.mpileup
!java -jar /home/belsawan/Downloads/VarScan.v2.4.0.jar mpileup2snp strain1_genome.mpileup \
--min-var-freq 0.001 --variants --output-vcf 1 > varscan_strain1_01_results.vcf
!cat varscan_strain1_01_results.vcf | gawk 'NR>24 {print "Mutations:",  $1, $2, $4, $5, "| Freq in percent:", substr($10, 26, 5)}'

[mpileup] 1 samples in 1 input files
<mpileup> Set max per-file depth to 8000
Only SNPs will be reported
Min coverage:	8
Min reads2:	2
Min var freq:	0.001
Min avg qual:	15
P-value thresh:	0.01
Reading input from strain1_genome.mpileup
1665 bases in pileup file
28 variant positions (28 SNP, 0 indel)
1 were failed by the strand-filter
27 variant positions reported (27 SNP, 0 indel)
Mutations: KF848938.1 125 T C | Freq in percent: 0.43%
Mutations: KF848938.1 129 T C | Freq in percent: 0.41%
Mutations: KF848938.1 139 T C | Freq in percent: 0.31%
Mutations: KF848938.1 151 A G | Freq in percent: 0.28%
Mutations: KF848938.1 276 A G | Freq in percent: 0.3%:
Mutations: KF848938.1 370 A G | Freq in percent: 0.27%
Mutations: KF848938.1 389 T C | Freq in percent: 0.26%
Mutations: KF848938.1 409 T C | Freq in percent: 0.3%:
Mutations: KF848938.1 566 A G | Freq in percent: 0.32%
Mutations: KF848938.1 595 G T | Freq in percent: 0.29%
Mutations: KF848938.1 722 A G | Freq in percent: 0.3%:
Mutations: K

In [112]:
# For strain 2

!samtools mpileup -f reference.fasta alignment_sorted_strain2.bam > strain2_genome.mpileup
!java -jar /home/belsawan/Downloads/VarScan.v2.4.0.jar mpileup2snp strain2_genome.mpileup \
--min-var-freq 0.001 --variants --output-vcf 1 > varscan_strain2_01_results.vcf
!cat varscan_strain2_01_results.vcf | gawk 'NR>24 {print "Mutations:",  $1, $2, $4, $5, "| Freq in percent:", substr($10, 26, 5)}'

[mpileup] 1 samples in 1 input files
<mpileup> Set max per-file depth to 8000
Only SNPs will be reported
Min coverage:	8
Min reads2:	2
Min var freq:	0.001
Min avg qual:	15
P-value thresh:	0.01
Reading input from strain2_genome.mpileup
1665 bases in pileup file
24 variant positions (24 SNP, 0 indel)
1 were failed by the strand-filter
23 variant positions reported (23 SNP, 0 indel)
Mutations: KF848938.1 125 T C | Freq in percent: 0.54%
Mutations: KF848938.1 158 A G | Freq in percent: 0.29%
Mutations: KF848938.1 222 T C | Freq in percent: 0.26%
Mutations: KF848938.1 235 T C | Freq in percent: 0.26%
Mutations: KF848938.1 291 T C | Freq in percent: 0.3%:
Mutations: KF848938.1 319 T C | Freq in percent: 0.26%
Mutations: KF848938.1 370 A G | Freq in percent: 0.27%
Mutations: KF848938.1 499 A G | Freq in percent: 0.27%
Mutations: KF848938.1 566 A G | Freq in percent: 0.28%
Mutations: KF848938.1 609 A G | Freq in percent: 0.27%
Mutations: KF848938.1 722 A G | Freq in percent: 0.28%
Mutations: K

In [113]:
# For strain 3

!samtools mpileup -f reference.fasta alignment_sorted_strain3.bam > strain3_genome.mpileup
!java -jar /home/belsawan/Downloads/VarScan.v2.4.0.jar mpileup2snp strain3_genome.mpileup \
--min-var-freq 0.001 --variants --output-vcf 1 > varscan_strain3_01_results.vcf
!cat varscan_strain3_01_results.vcf | gawk 'NR>24 {print "Mutations:",  $1, $2, $4, $5, "| Freq in percent:", substr($10, 26, 5)}'

[mpileup] 1 samples in 1 input files
<mpileup> Set max per-file depth to 8000
Only SNPs will be reported
Min coverage:	8
Min reads2:	2
Min var freq:	0.001
Min avg qual:	15
P-value thresh:	0.01
Reading input from strain3_genome.mpileup
1665 bases in pileup file
27 variant positions (27 SNP, 0 indel)
1 were failed by the strand-filter
26 variant positions reported (26 SNP, 0 indel)
Mutations: KF848938.1 105 A G | Freq in percent: 0.94%
Mutations: KF848938.1 139 T C | Freq in percent: 0.36%
Mutations: KF848938.1 158 A G | Freq in percent: 0.36%
Mutations: KF848938.1 165 T C | Freq in percent: 0.34%
Mutations: KF848938.1 235 T C | Freq in percent: 0.37%
Mutations: KF848938.1 276 A G | Freq in percent: 0.38%
Mutations: KF848938.1 370 A G | Freq in percent: 0.27%
Mutations: KF848938.1 414 T C | Freq in percent: 0.32%
Mutations: KF848938.1 421 A G | Freq in percent: 0.26%
Mutations: KF848938.1 566 A G | Freq in percent: 0.4%:
Mutations: KF848938.1 660 A G | Freq in percent: 0.32%
Mutations: K